## Predicting Heart Disease Using Machine Learning

This notebook looks into using various Python-based machine learning and data science libraries in an attempt to build a machine learning model capable of predicting whether or not someone has heart disease based on their medical attributes.

We're going to take the following approach:
1. Problem definition
2. Data
3. Evaluation
4. Features
5. Modeling
6. Experimentation
___

### 1. Problem Definition
In a statement,
> Given clinical parameters about a patient, can we predict whether or not they have heart disease?

### 2. Data
The original data came from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Heart+Disease).

There is also a version of it available on [Kaggle](https://www.kaggle.com/ronitf/heart-disease-uci).

### 3. Evaluation
>If we can reach 95% accuracy at predicting whether or not a patient had heart disease during the proof of concept, we'll pursue the project.

### 4. Features
This is where you'll get different information about each of the features in your data. You can do this via doing your own research or by talking to a subject matter expert.

**Create a data dictionary**
1. age - age in years
2. sex - (1 = male; 0 = female)
3. cp - chest pain type
    * 0: Typical angina: chest pain related decrease blood supply to the heart
    * 1: Atypical angina: chest pain not related to heart
    * 2: Non-anginal pain: typically esophageal spasms (non heart related)
    * 3: Asymptomatic: chest pain not showing signs of disease
4. trestbps - resting blood pressure (in mm Hg on admission to the hospital) anything above 130-140 is typically cause for concern
5. chol - serum cholestoral in mg/dl
    * serum = LDL + HDL + .2 * triglycerides
    * above 200 is cause for concern
6. fbs - (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
    * '>126' mg/dL signals diabetes
7. restecg - resting electrocardiographic results
    * 0: Nothing to note
    * 1: ST-T Wave abnormality
        * can range from mild symptoms to severe problems
        * signals non-normal heart beat
    * 2: Possible or definite left ventricular hypertrophy
        * Enlarged heart's main pumping chamber
8. thalach - maximum heart rate achieved
9. exang - exercise induced angina (1 = yes; 0 = no)
10. oldpeak - ST depression induced by exercise relative to rest looks at stress of heart during excercise unhealthy heart will stress more
11. slope - the slope of the peak exercise ST segment
    * 0: Upsloping: better heart rate with excercise (uncommon)
    * 1: Flatsloping: minimal change (typical healthy heart)
    * 2: Downslopins: signs of unhealthy heart
12. ca - number of major vessels (0-3) colored by flourosopy
    * colored vessel means the doctor can see the blood passing through
    * the more blood movement the better (no clots)
13. thal - thalium stress result
    * 1,3: normal
    * 6: fixed defect: used to be defect but ok now
    * 7: reversable defect: no proper blood movement when excercising
14. target - have disease or not (1=yes, 0=no) (= the predicted attribute)

## Preparing the tools
We're going to use pandas, Matplotlib and NumPy for data analysis and manipulation.

In [ ]:
# Import all the tools we need

# Regular EDA (exploratory data analysis) and plotting libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# We want our plots to appear inside the notebook
%matplotlib inline

# Models from Scikit-Learn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Model Evaluations
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import plot_roc_curve

## Load Data

In [ ]:
df = pd.read_csv('heart-disease.csv')
df.shape # (rows, columns)

## Data Exploration (exploratory data analysis or EDA)
The goal here is to find our more about the data and become a subject matter expert on the dataset you're working with.
1. What question(s) are you trying to solve?
2. What kind of data do we have and how do we treat different types?
3. What's missing from the data and how do you deal with it?
4. Where are the outliers and why should you care about them?
5. How can you add, change or remove features to get more out of your data?

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
# Let's find out how many of each class there are
df['target'].value_counts()

Recall from our data dictionary...

`target - have disease or not (1=yes, 0=no) (= the predicted attribute)`

\
We can see that this is a relatively balanced problem; meaning that we have a similar number of examples in both classes.

In [ ]:
# Let's visualize this
df['target'].value_counts().plot(kind='bar', color=['salmon', 'lightblue']);

In [ ]:
# Next, we may want to look at different information about our dataframe
df.info()

In [ ]:
# Are there any missing values?
df.isna().sum()

In [ ]:
# If we still want to find out more about our dataframe
df.describe()

There is not too much that we can make use of for now.

So far we have some quick insights into the data, but no real structure.

We want to gain intuition into how the features relate to the target variable.

Lets try to compare different columns to get an idea of where the patterns are within our data.

### Heart Disease Frequency according to Sex
* target - have disease or not (1=yes, 0=no) (= the predicted attribute)  
* sex - (1 = male; 0 = female)

In [ ]:
df.sex.value_counts()

In [ ]:
# Compare target column with sex column
pd.crosstab(df.target, df.sex)

In [ ]:
# We can infer from the above data the relative amounts of heart disease by sex
print(f'Women with heart disease: {72/96*100}%\nMen with heart disease: {93/207*100}%\nAnyone with heart disease: {(72+93)/303*100}%')

In [ ]:
# Create a plot of crosstab
pd.crosstab(df.target, df.sex).plot(kind='bar',
                                    figsize=(10, 6),
                                    color=['salmon', 'lightblue'])
plt.title('Heart Disease Frequency for Sex')
plt.xlabel('0 = No Disease, 1 = Disease')
plt.ylabel('Amount')
plt.legend(['Female', 'Male'])
plt.xticks(rotation=0);                           

### Age vs. Max Heart Rate for Heart Disease
* target - have disease or not (1=yes, 0=no) (= the predicted attribute)
* age - age in years
* thalach - maximum heart rate achieved

In [ ]:
df['thalach'].value_counts()

There are 91 different values in the thalach column.

We may not want to visualize it on a bar graph.

In [ ]:
# Create another figure
plt.figure(figsize=(10, 6))

# Scatter with positive examples
plt.scatter(df.age[df.target==1],
            df.thalach[df.target==1],
            c='salmon')

# Scatter with negative examples
plt.scatter(df.age[df.target==0],
            df.thalach[df.target==0],
            c='lightblue')
            
# Add some helpful info
plt.title('Heart Disease in function of Age and Max Heart Rate')
plt.xlabel('Age')
plt.ylabel('Max Heart Rate')
plt.legend(['Disease', 'No Disease']);

In [ ]:
# Check the distribution of the age column with a histogram
df.age.plot.hist();

### Heart Disease Frequency per Chest Pain Type
* target - have disease or not (1=yes, 0=no) (= the predicted attribute)
* cp - chest pain type
    * 0: Typical angina: chest pain related decrease blood supply to the heart
    * 1: Atypical angina: chest pain not related to heart
    * 2: Non-anginal pain: typically esophageal spasms (non heart related)
    * 3: Asymptomatic: chest pain not showing signs of disease

In [ ]:
pd.crosstab(df.cp, df.target)

In [ ]:
# Make the crosstab more visual
pd.crosstab(df.cp, df.target).plot(kind='bar',
                                   figsize=(10, 6),
                                   color=['lightblue', 'salmon'])

# Add some communication
plt.title('Heart Disease Frequency Per Chest Pain Type')
plt.xlabel('Chest Pain Type')
plt.ylabel('Amount')
plt.legend(['No Disease', 'Disease'])
plt.xticks(rotation=0);

# Save and show
fig.savefig('images/heart-disease-frequency-per-cp-type.png')

Why does heart disease increase with non-heart related pain?

We may want to consult a subject matter expert.

### Correlation Matrix
* Correlation matrix - big table of numbers telling us how our independent variables and our dependent variable are related to one another.

In [ ]:
# independent variables - feature labels (age, sex, cp, ..., thal)
# dependent variable - target
df.head()

In [ ]:
# Make a correlation matrix
df.corr()

In [ ]:
# Let's make our correlation matrix a little prettier
corr_matrix = df.corr()
fig, ax = plt.subplots(figsize=(15, 10))
ax = sns.heatmap(corr_matrix,
                 annot=True,
                 linewidths=0.5,
                 fmt='.2f',
                 cmap='YlGnBu');

# Correction for older version of matplotlib (3.1.1); need to adjust y-axis limitations
# bottom, top = ax.get_ylim()
# ax.set_ylim(bottom + 0.5, top - 0.5)                 

**How is chest pain related to the target column?**

There is a positive value of 0.43, indicating that there is a positive correlation. Therefore, as cp goes up, the target value also increases.

We can investigate this by looking at our bar graph...
<img src='images/heart-disease-frequency-per-cp-type.png' alt='' height=350>